In [1]:
import pandas as pd
import numpy as np
import astropy as ap
from astropy import units as u
from astropy.coordinates import SkyCoord
import kcorrect
import kcorrect.utils as ut
from astropy.cosmology import FlatLambdaCDM
import os
import matplotlib.pyplot as plt

Defining Cosmology

In [2]:
cosmo=FlatLambdaCDM(H0=70,Om0=0.3)

Reading in Catalogs (LAMBDAR and TASCA)

In [3]:
print('Reading in Catalogs')
COSMOS_PHOT_LAMBDAR=pd.read_csv('/Users/lucashunt/ASTRODATA/LCBG_LUMINOSITY_FUNCTION/COSMOS_CATALOGS/Photometry/G10CosmosLAMBDARCatv05.csv')
TASCA_COSMOS_MORPH=pd.read_csv('/Users/lucashunt/ASTRODATA/LCBG_LUMINOSITY_FUNCTION/COSMOS_CATALOGS/Morphology/MORPH.tsv',delim_whitespace=True,header=53,dtype=float,error_bad_lines=False)


Reading in Catalogs


Setting Variables

In [4]:
kcordir=os.environ["KCORRECT_DIR"]

*********************************************
Matching Catalogs
*********************************************

In [5]:
print('Matching Catalogs')

Matching Catalogs


Generating Astropy skycoord objects to easily match catalogs based on position

In [6]:
TASCA_COORD=SkyCoord(ra=TASCA_COSMOS_MORPH['RAJ2000'].values*u.degree,dec=TASCA_COSMOS_MORPH['DEJ2000'].values*u.degree)
G10_COORD=SkyCoord(ra=COSMOS_PHOT_LAMBDAR['RA'].values*u.degree,dec=COSMOS_PHOT_LAMBDAR['DEC'].values*u.degree)

Matching Catalogs

In [7]:
idx,d2d,d3d=G10_COORD.match_to_catalog_sky(TASCA_COORD)
COSMOS_PHOT_LAMBDAR['Rh']=TASCA_COSMOS_MORPH['Rh'][idx].values
COSMOS_PHOT_LAMBDAR['separation']=d2d.arcsecond
COSMOS_PHOT_LAMBDAR.loc[COSMOS_PHOT_LAMBDAR.separation>1,'Rh']=np.nan
COSMOS_FLUXES=COSMOS_PHOT_LAMBDAR

*********************************************
Calculating apparent magnitudes
*********************************************

In [8]:
print('Calculating apparent magnitudes')

Calculating apparent magnitudes


Determining what data is in each column

In [9]:
column_headers=COSMOS_FLUXES.columns.values
telescope_flux_headers=[s for s in column_headers if (('subaru' in s) or ('uvista' in s) or ('galex' in s) or ('cfht' in s) or ('irac' in s) or ('mips' in s) or ('pacs' in s) or ('spire' in s)) and ('err' not in s)]

Calculating magnitudes (And creating column for maggies, see kcorrect.org for explanation)

In [10]:
for i in telescope_flux_headers:
    COSMOS_FLUXES['mag_'+i]=-2.5*np.log10(COSMOS_FLUXES[i]/3631)
    COSMOS_FLUXES['mag_'+i+'_err']=2.5/np.log(10)*COSMOS_FLUXES[i+'_err']/COSMOS_FLUXES[i]
    COSMOS_FLUXES['maggies_'+i]=np.nan
    COSMOS_FLUXES['invervar_'+i]=np.nan

COSMOS_FLUXES.loc[COSMOS_FLUXES.isnull().mag_galex_fuv,'mag_galex_fuv_err']=np.nan
COSMOS_FLUXES.loc[COSMOS_FLUXES.isnull().mag_galex_nuv,'mag_galex_nuv_err']=np.nan

/Users/lucashunt/programs/anaconda3/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log10
  
/Users/lucashunt/programs/anaconda3/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in log10
  


In [23]:
telescope_flux_headers

['galex_fuv',
 'galex_nuv',
 'cfht_u',
 'subaru_B',
 'subaru_V',
 'subaru_g',
 'subaru_r',
 'subaru_i',
 'subaru_z',
 'uvista_Y',
 'uvista_J',
 'uvista_H',
 'uvista_K',
 'irac_1',
 'irac_2',
 'irac_3',
 'irac_4',
 'mips_24',
 'mips_70',
 'pacs_100',
 'pacs_160',
 'spire_250',
 'spire_350',
 'spire_500',
 'subaru_ia427',
 'subaru_ia464',
 'subaru_ia484',
 'subaru_ia505',
 'subaru_ia527',
 'subaru_ia574',
 'subaru_ia624',
 'subaru_ia679',
 'subaru_ia709',
 'subaru_ia738',
 'subaru_ia767',
 'subaru_ia827',
 'subaru_nb711',
 'subaru_nb816']

*********************************************
Getting k-correction
*********************************************

Making columns for kcorrect output values

In [18]:
numbers=np.arange(1,7,1)
for string in ['c'+str(number) for number in numbers]:
    COSMOS_FLUXES[string]=np.nan

Making list that contains filters used for k-correction, making columns for synthetic filters. 

Converting to maggies and invervar

In [19]:
column_headers=COSMOS_FLUXES.columns.values

In [20]:
print('Converting to maggies')
column_headers=COSMOS_FLUXES.columns.values
for column in column_headers:
    if 'maggies' in column:
        COSMOS_FLUXES[column]=ut.mag2maggies(COSMOS_FLUXES['mag_'+column.split('maggies_')[1]])
        COSMOS_FLUXES['invervar_'+column.split('maggies_')[1]]=ut.invariance(COSMOS_FLUXES[column],COSMOS_FLUXES['mag_'+column.split('maggies_')[1]+'_err'])

Converting to maggies


Here we make a list of the filters we want to use for the k-correct code

In [21]:
kcorr_filts=['cfht_u','subaru_B','subaru_V','subaru_r','subaru_i','subaru_z']
for f in kcorr_filts:
    COSMOS_FLUXES[f+'_synthetic']=np.nan
    COSMOS_FLUXES[f+'0_synthetic']=np.nan

Loading Filters

In [22]:
kcorrect.load_templates()
kcorrect.load_filters(kcordir+'/data/templates/Lum_Func_Filters_US.dat')

Creating an index array so that we step through each source to calculate the coefficients needed for k-correct

In [16]:
cmm_ind=COSMOS_FLUXES.index.values

In [17]:
for i in cmm_ind:
    COSMOS_FLUXES.loc[i,'c1':'c6']=kcorrect.fit_nonneg(np.array(COSMOS_FLUXES.loc[i,'Z_BEST'],dtype=float),np.array(COSMOS_FLUXES.loc[i,['maggies_'+col for col in kcorr_filts]],dtype=float),np.array(COSMOS_FLUXES.loc[i,['invervar_'+col for col in kcorr_filts]],dtype=float))

KeyboardInterrupt: 

Recalculating the "maggies" for each object based on the best guess of the SED

In [ ]:
for i in cmm_ind:
    COSMOS_FLUXES.loc[i,[f+'_synthetic' for f in kcorr_filts]]=kcorrect.reconstruct_maggies(COSMOS_FLUXES.loc[i,'c1':'c6'])[1:]
    COSMOS_FLUXES.loc[i,[f+'0_synthetic' for f in kcorr_filts]]=kcorrect.reconstruct_maggies(COSMOS_FLUXES.loc[i,'c1':'c6'],redshift=0)[1:]

Converting the synthetic maggies to apparent magnitude

In [ ]:
COSMOS_FLUXES[[f+'_synthetic_mag' for f in kcorr_filts]]=-2.5*np.log10(COSMOS_FLUXES[[f+'_synthetic' for f in kcorr_filts]])

Calculating Johnson U/B/V from the SED

In [ ]:
COSMOS_FLUXES['U0_synthetic']=np.nan
COSMOS_FLUXES['B0_synthetic']=np.nan
COSMOS_FLUXES['V0_synthetic']=np.nan

In [ ]:
kcorrect.load_templates()
kcorrect.load_filters(kcordir+'/data/templates/bessell_ubv.dat')

In [ ]:
for i in cmm_ind:
    COSMOS_FLUXES.loc[i,['U0_synthetic','B0_synthetic','V0_synthetic']]=kcorrect.reconstruct_maggies(COSMOS_FLUXES.loc[i,'c1':'c6'],redshift=0)[1:]

Converting to magnitudes and then adding offsets to go from AB magnitudes to Vega magnitudes

In [ ]:
COSMOS_FLUXES[['U0_synthetic_mag','B0_synthetic_mag','V0_synthetic_mag']]=-2.5*np.log10(COSMOS_FLUXES[['U0_synthetic','B0_synthetic','V0_synthetic']])
COSMOS_FLUXES['U0_synthetic_mag']=COSMOS_FLUXES['U0_synthetic_mag']-0.79
COSMOS_FLUXES['B0_synthetic_mag']=COSMOS_FLUXES['B0_synthetic_mag']+0.09
COSMOS_FLUXES['V0_synthetic_mag']=COSMOS_FLUXES['V0_synthetic_mag']-0.02

Actually calculating k-correction from Subaru filters to Bessell Filters

In [ ]:
COSMOS_FLUXES[[f+'_kcorr_B' for f in kcorr_filts]]=-2.5*np.log10(COSMOS_FLUXES[[f+'_synthetic' for f in kcorr_filts]]/np.stack((COSMOS_FLUXES['B0_synthetic'],COSMOS_FLUXES['B0_synthetic'],COSMOS_FLUXES['B0_synthetic'],COSMOS_FLUXES['B0_synthetic'],COSMOS_FLUXES['B0_synthetic'],COSMOS_FLUXES['B0_synthetic']),axis=-1))

Calculating B-V color

In [ ]:
COSMOS_FLUXES['rest_frame_B-V']=COSMOS_FLUXES['B0_synthetic_mag']-COSMOS_FLUXES['V0_synthetic_mag']

Calculating Absolute Magnitude using k-correction (Different cells below cover different redshift ranges)

In [ ]:
COSMOS_FLUXES['Abs_B_Mag']=np.nan

In [ ]:
COSMOS_FLUXES.loc[COSMOS_FLUXES.Z_BEST<0.1,'Abs_B_Mag']=COSMOS_FLUXES.loc[COSMOS_FLUXES.Z_BEST<0.1,'mag_subaru_B']-0.05122-cosmo.distmod(COSMOS_FLUXES.loc[COSMOS_FLUXES.Z_BEST<0.1,'Z_BEST']).value-COSMOS_FLUXES.loc[COSMOS_FLUXES.Z_BEST<0.1,'subaru_B_kcorr_B']+0.09

In [ ]:
COSMOS_FLUXES.loc[(COSMOS_FLUXES.Z_BEST>0.1)&(COSMOS_FLUXES.Z_BEST<0.35),'Abs_B_Mag']=COSMOS_FLUXES.loc[(COSMOS_FLUXES.Z_BEST>0.1)&(COSMOS_FLUXES.Z_BEST<0.35),'mag_subaru_V']-0.069802-cosmo.distmod(COSMOS_FLUXES.loc[(COSMOS_FLUXES.Z_BEST>0.1)&(COSMOS_FLUXES.Z_BEST<0.35),'Z_BEST']).value-COSMOS_FLUXES.loc[(COSMOS_FLUXES.Z_BEST>0.1)&(COSMOS_FLUXES.Z_BEST<0.35),'subaru_V_kcorr_B']+0.09

In [ ]:
COSMOS_FLUXES.loc[(COSMOS_FLUXES.Z_BEST>0.35)&(COSMOS_FLUXES.Z_BEST<0.55),'Abs_B_Mag']=COSMOS_FLUXES.loc[(COSMOS_FLUXES.Z_BEST>0.35)&(COSMOS_FLUXES.Z_BEST<0.55),'mag_subaru_r']-0.01267-cosmo.distmod(COSMOS_FLUXES.loc[(COSMOS_FLUXES.Z_BEST>0.35)&(COSMOS_FLUXES.Z_BEST<0.55),'Z_BEST']).value-COSMOS_FLUXES.loc[(COSMOS_FLUXES.Z_BEST>0.35)&(COSMOS_FLUXES.Z_BEST<0.55),'subaru_r_kcorr_B']+0.09

In [ ]:
COSMOS_FLUXES.loc[(COSMOS_FLUXES.Z_BEST>0.55)&(COSMOS_FLUXES.Z_BEST<0.75),'Abs_B_Mag']=COSMOS_FLUXES.loc[(COSMOS_FLUXES.Z_BEST>0.55)&(COSMOS_FLUXES.Z_BEST<0.75),'mag_subaru_i']-0.004512-cosmo.distmod(COSMOS_FLUXES.loc[(COSMOS_FLUXES.Z_BEST>0.55)&(COSMOS_FLUXES.Z_BEST<0.75),'Z_BEST']).value-COSMOS_FLUXES.loc[(COSMOS_FLUXES.Z_BEST>0.55)&(COSMOS_FLUXES.Z_BEST<0.75),'subaru_i_kcorr_B']+0.09

In [ ]:
COSMOS_FLUXES.loc[COSMOS_FLUXES.Z_BEST>0.75,'Abs_B_Mag']=COSMOS_FLUXES.loc[COSMOS_FLUXES.Z_BEST>0.75,'mag_subaru_z']-0.00177-cosmo.distmod(COSMOS_FLUXES.loc[COSMOS_FLUXES.Z_BEST>0.75,'Z_BEST']).value-COSMOS_FLUXES.loc[COSMOS_FLUXES.Z_BEST>0.75,'subaru_z_kcorr_B']+0.09

Calculate absolute magnitude from the "Synthetic" apparent magnitude

In [ ]:
COSMOS_FLUXES['Abs_B_Mag_synthetic']=COSMOS_FLUXES['B0_synthetic_mag']-cosmo.distmod(COSMOS_FLUXES.Z_BEST.values).value

Diagnostic plotting

In [ ]:
plt.plot(COSMOS_FLUXES.loc[COSMOS_FLUXES.Z_BEST<1.0,'Abs_B_Mag'],COSMOS_FLUXES.loc[COSMOS_FLUXES.Z_BEST<1.0,'Abs_B_Mag']-COSMOS_FLUXES.loc[COSMOS_FLUXES.Z_BEST<1.0,'Abs_B_Mag_synthetic'],'.')
plt.plot(COSMOS_FLUXES.loc[(COSMOS_FLUXES.Z_BEST<1.0)&(COSMOS_FLUXES.Z_USE<3)&(COSMOS_FLUXES.SG_MASTER==0),'Abs_B_Mag'],COSMOS_FLUXES.loc[(COSMOS_FLUXES.Z_BEST<1.0)&(COSMOS_FLUXES.Z_USE<3)&(COSMOS_FLUXES.SG_MASTER==0),'Abs_B_Mag']-COSMOS_FLUXES.loc[(COSMOS_FLUXES.Z_BEST<1.0)&(COSMOS_FLUXES.Z_USE<3)&(COSMOS_FLUXES.SG_MASTER==0),'Abs_B_Mag_synthetic'],'.')
print(len(COSMOS_FLUXES.loc[COSMOS_FLUXES.Z_BEST>0.1(COSMOS_FLUXES.Z_BEST<1.0)&(COSMOS_FLUXES.Z_USE<3)&(COSMOS_FLUXES.SG_MASTER==0),'Abs_B_Mag']))
plt.hlines(1,-30,200)
plt.hlines(-1,-30,200)

In [ ]:
plt.plot(COSMOS_FLUXES.loc[(COSMOS_FLUXES.Z_BEST<1.0)&(COSMOS_FLUXES.Z_USE<3)&(COSMOS_FLUXES.SG_MASTER==0),'Abs_B_Mag'],'.')


Calculate surface brightness

In [ ]:
COSMOS_FLUXES['Surface_Brightness_B']=COSMOS_FLUXES.Abs_B_Mag+2.5*np.log10((2*np.pi*np.power(cosmo.angular_diameter_distance(COSMOS_FLUXES.Z_BEST.values).value*np.tan(COSMOS_FLUXES.Rh.values*0.03*4.84814e-6)*(814/(445*(1+COSMOS_FLUXES.Z_BEST.values)))**0.108*1e3,2)))+2.5*np.log10((360*60*60/(2*np.pi*0.01))**2)

Calculate effective radius and determine whether each source is an LCBG

In [ ]:
COSMOS_FLUXES['R_eff_arcsec_F814W']=COSMOS_FLUXES.loc[:,'Rh']*0.03
COSMOS_FLUXES['R_eff_arcsec_B']=COSMOS_FLUXES.loc[:,'Rh']*0.03*(814/(445*(1+COSMOS_FLUXES.Z_BEST.values)))**0.108
COSMOS_FLUXES['is_LCBG']=0
COSMOS_FLUXES.loc[(COSMOS_FLUXES.Abs_B_Mag.values<-18.5)&(COSMOS_FLUXES.Surface_Brightness_B.values<21)&(COSMOS_FLUXES['rest_frame_B-V'].values<0.6),'is_LCBG']=1

In [ ]:
plt.plot(COSMOS_FLUXES['rest_frame_B-V'],COSMOS_FLUXES['Surface_Brightness_B'],'.')

In [ ]:
COSMOS_FLUXES.to_csv('/Users/lucashunt/ASTRODATA/LCBG_LUMINOSITY_FUNCTION/COSMOS_CATALOGS/Photometry/COSMOS_CONVERTED_CATALOG.csv')